In [19]:
import tensorflow as tf

In [20]:
# Dense층 직접 만들기

class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        self.w = tf.Variable(tf.random.uniform(shape=(input_size, output_size), minval=0, maxval=0.1))
        self.b = tf.Variable(tf.zeros(shape=(output_size, )))
        
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.w)) + self.b
    
    @property 
    def weights(self):
        return [self.w, self.b]  # 리스트를 묶어서 리턴
    # @는 일종의 예약어 취급하기. 이건 특별한 약속을 하고 있다. 
    # 파이썬에서는 decoration이라고 하고 원래 함수는 weights( ) 이렇게 불렀는데
    # 이건 함수가 아니라 속성을 말하며 weights는 괄호를 안붙이고 부른다.

In [21]:
# Sequential 만들기

class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers   # 왼쪽은 새로 만들어지는 변수이다. layers를 받아서 새로운 변수에 넣어줌
        
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights  # layers들을 순환하면서 가중치를 업데이트 시켜준다.
        return weights

In [22]:
# 모델 구성하기
model = NaiveSequential([
            NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
            NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
        ])
# 512이 인 이유는 위에 512를 받아서 하는거여서.

In [25]:
assert len(model.weights) == 4
# assert는 중간중간에 확인을 하라는 건데 4면 계속 진행하고 아니면 멈춰라 라는것.
# 왜 4가 나올까? 

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras import Sequential #대문자라는 뜻은 클래스 라는 뜻
from keras import optimizers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from keras.utils.np_utils import to_categorical

import keras
from keras import layers
from keras.datasets import mnist

In [39]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32')/255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32')/255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [51]:
# 이미지 처리하는 부분

inputs = Input(shape=(28,28,1 ))
x = Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
                       # kernel_size는 가로, 세로가 3이라는뜻(3*3)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=64, kernel_size=3, activation='relu')(inputs)
x = MaxPooling2D(pool_size=2)(x)
x = Conv2D(filters=128, kernel_size=3, activation='relu')(inputs)
x = Flatten()(x)  # 괄호가 있다는건 함수가 아니라 클래스라는 표시이다.
                  # 함수였다면 Flatten(x) 라고 썼어야 했다.

x = Dense(20, activation='relu')(x)
x = Dense(20, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs)

In [48]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [49]:
history = model.fit(train_images, train_labels,
                   epochs=2, batch_size=256)

Epoch 1/2
235/235 [==============================] - 37s 154ms/step - loss: 0.4118 - accuracy: 0.8760
Epoch 2/2
235/235 [==============================] - 36s 153ms/step - loss: 0.1090 - accuracy: 0.9686


In [50]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 3s 7ms/step - loss: 0.1236 - accuracy: 0.9594


[0.1236100122332573, 0.9593999981880188]

In [ ]:
앞에 했던건 이미지 데이터이다.
이 이미지 데이터는 숫자 배열로 되어있는데 이건 이미지를 반영하는 이미지 숫자이다.(28, 28,1)
가로세로 28짜리 흑백(1) 사진이라는것이다.

이미지 숫자(28, 28, 1)  >>> 숫자 >>> 1
이미지를 1로 읽은게 아니라 이미지를 그럴듯하게 변영한 다음에 한건
DNN은 숫자를 보고 1이라고 맞추고
      숫자 -> 1(레이어 분류)
CNN은 이미지 처리를 한 후 그 다음 처리 된 것을 DNN의 원리를 그대로 갖다가 쓴다.
      이미지 -> DNN

In [52]:
ConV  (32,(3*3))
32는 필터이다. 즉 32장의 필터가 있는거다.
(3*3)은 가로세로 크기를 말한다.

피쳐맵 = 사진
예) 5*5 사진
필터 = 커널 = 윈도우 
예) 고양이 코, 3*3

피처맵에 필터를 덮는다. 패치=patch
패치나 필터나 크기는 똑같다.
사진에서 여러개의 패치가 만들어진다.

고양이 코가 있는지 없는지 확을 위해
사진에 패치를 덮은 상태에서 한칸씩 움직인다.
한칸씩 움직이는건 디폴트 값이다.

한칸씩 가다보면 첫번째, 두번째, 세번째까지 가면 더 못간다.
그러고 나면 한칸 밑으로 내려가서 다시 옆으로 또 한칸씩 간다.
이런식으로 움직이면서 필터가 고양이 코가 있는지 없는지 확인한다.

방금 가로세로가 5 바이 5이고 필터가 3 바이 3짜리인데
움직이면 옆으로 총 3칸 밑으로 3칸을 움직였다.
만약 5가 아니라 7 바이 7이고 필터는 3 바이 3이면
옆으로 밑으로 몇 칸씩 가게 될까?
답은 5칸을 가게 된다.(옆, 아래로 각각 5칸)
7 바이 7에서 필터를 대고 움직이면 끄트머리는 5개가 나온다.(가로로 봤을때) p290

SyntaxError: unmatched ')' (3391952540.py, line 2)

In [ ]:
원래는 5 바이 5였지만
줄어든느것을 막기 위해
겉에 0으로 감싼다.
그 상태로 컨블루션을 시키는데 0으로 둘러쌓은것부터 시작한다.
그럼 7바이 7은 줄어봤자 결과물이 5바이 5가 된다.
2개의 로우와 칼럼을 더 넣어주는것을 패딩이라고 한다.

패딩이 무엇인지 왜 하는지를 알아야한다.